# 🔹 Qué hace este bloque
- Recorre todas las categorías.

In [82]:
import requests
from bs4 import BeautifulSoup
import time # Para no hacer pedidos(requests) una tras otra, porque puede causar problemas


# Defino la base del sitio principal
BASE_URL = "https://books.toscrape.com/"

# Creo la URL inicial del sitio y uso BASE_URL + url_relativa para armar URLs completas de las categorías
CATEGORIAS_URL = BASE_URL + "index.html" 

def extraer_categorias():
    # Extraigo con el método get las categorias y con el "parser" obtengo la info relevante
    respuesta = requests.get(CATEGORIAS_URL)
    soup = BeautifulSoup(respuesta.text, "html.parser")

    # Extaigo los enlaces de categorias (hay uno para cada categoria dentro de 'ul.nav-list')
    categorias_links = soup.select("ul.nav-list ul li a")

    categoria = {} # Creo un diccionario vacio para guardar categorias

    for link in categorias_links: 
        nombre = link.text.strip() # Obtengo y limpio el nombre de la categoria
        url_relativa = link['href'] # Saco la URL relativa del href
        url_completa = BASE_URL + url_relativa # Armo la URL completa
        categoria[nombre] = url_completa # Guardo en el diccionario: nombre -> URL

    print(f"Categorias encontradas: {len(categoria)}\n")
    return categoria
# Recorre cada par del diccionario : (clave,valor) y con el .items devuelve la lista de pares
    for nombre, url in categoria.items():
        print(f"- {nombre}: {url}")
categoria = extraer_categorias()

Categorias encontradas: 50



# Creación de Base de Datos

In [69]:
import sqlite3

# Crear conexión y cursor
conn = sqlite3.connect("libros.db") # Crea el archivo libros.db
cursor = conn.cursor()

cursor.execute("""
               
CREATE TABLE IF NOT EXISTS categorias(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT UNIQUE
)
""")

cursor.execute("""
               
CREATE TABLE IF NOT EXISTS autores(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT UNIQUE
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS libros(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT,
    precio REAL,
    rating TEXT,
    disponibilidad TEXT,
    link TEXT,
    UPC TEXT,
    categoria_id INTEGER,
    descripcion TEXT,
    FOREIGN KEY (categoria_id) REFERENCES categorias(id)                                
)
""")
try:
     cursor.execute("ALTER TABLE libros ADD COLUMN categoria_id TEXT")
except sqlite3.OperationalError:
     print("La columna 'categoria_id' ya existe.")

# try:
#     cursor.execute("ALTER TABLE libros ADD COLUMN disponibilidad TEXT")
# except sqlite3.OperationalError:
#     print("La columna 'disponibilidad' ya existe.")


cursor.execute("""
CREATE TABLE IF NOT EXISTS libro_autor(
    libro_id INTEGER,
    autor_id INTEGER,
    PRIMARY KEY (libro_id, autor_id),
    FOREIGN KEY (libro_id) REFERENCES libros(id),
    FOREIGN KEY (autor_id) REFERENCES autores(id)
)
""")
cursor.execute("PRAGMA table_info(libros)")
print(cursor.fetchall())   
conn.commit()
conn.close()

La columna 'categoria_id' ya existe.
[(0, 'id', 'INTEGER', 0, None, 1), (1, 'titulo', 'TEXT', 0, None, 0), (2, 'categoria', 'TEXT', 0, None, 0), (3, 'precio', 'REAL', 0, None, 0), (4, 'rating', 'TEXT', 0, None, 0), (5, 'descripcion', 'TEXT', 0, None, 0), (6, 'disponibilidad', 'TEXT', 0, None, 0), (7, 'link', 'TEXT', 0, None, 0), (8, 'UPC', 'TEXT', 0, None, 0), (9, 'categoria_id', 'TEXT', 0, None, 0)]


# Conexion a la BD.
- Insertar categorias
- Insertar autores
- Insertar libros
- Relación libros con autor


In [83]:
import sqlite3

# Conexión a la BD
def conectar():
    return sqlite3.connect("libros.db")

# Guardar categoría
def guardar_categoria(nombre):
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute("INSERT OR IGNORE INTO categorias(nombre) VALUES(?)", (nombre,))
    conn.commit()
    cursor.execute("SELECT id FROM categorias WHERE nombre=?", (nombre,))
    categoria_id = cursor.fetchone()[0]
    conn.close()
    return categoria_id

# Guardar autor
def guardar_autor(nombre):
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute("INSERT OR IGNORE INTO autores(nombre) VALUES(?)", (nombre,))
    conn.commit()
    cursor.execute("SELECT id FROM autores WHERE nombre=?", (nombre,))
    autor_id = cursor.fetchone()[0]
    conn.close()
    return autor_id

# Guardar libro
def guardar_libro(titulo, precio, rating, disponibilidad, link, UPC, categoria_id,descripcion):
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO libros(titulo, precio, rating, disponibilidad, link, UPC, categoria_id,descripcion)
        VALUES(?,?,?,?,?,?,?,?)
    """, (titulo, precio, rating, disponibilidad, link, UPC, categoria_id, descripcion))
    conn.commit()
    libro_id = cursor.lastrowid
    conn.close()
    return libro_id

# Relacionar libro con autor
def guardar_libro_autor(libro_id, autor_id):
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute("INSERT OR IGNORE INTO libro_autor(libro_id, autor_id) VALUES(?,?)", (libro_id, autor_id))
    conn.commit()
    conn.close()



# Consultas Google Books API

In [84]:
import requests
from bs4 import BeautifulSoup

# --- Scraping en BooksToScrape (por autor) ---
def obtener_autor_books_to_scrape(url_libro):
    try:
        response = requests.get(url_libro)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            info_table = soup.select('table.table.table-striped tr')
            if len(info_table) > 1:
                autor = info_table[1].text.strip()
                return autor
    except Exception as e:
        print(f"Error en scraping: {e}")
    return "Autor desconocido"


# --- API de Google Books ---
def buscar_libro(titulo):
    url = "https://www.googleapis.com/books/v1/volumes"
    params = {
        "q": f"intitle:{titulo}",
        "maxResults": 1,
        "langRestrict": "es"
    }

    respuesta = requests.get(url, params=params)

    if respuesta.status_code == 200:
        datos = respuesta.json()
        if "items" in datos:
            info = datos["items"][0]["volumeInfo"]

            titulo_libro = info.get("title", "Sin título")
            autores = ", ".join(info.get("authors", ["Autor desconocido"]))
            descripcion = info.get("description", "Sin descripción")

            return {
                "Título": titulo_libro,
                "Autores": autores,
                "Descripción": descripcion
            }

    # Si no encuentra en la API, intenta con BooksToScrape
    print("No se encontró en Google Books, intentando en BooksToScrape...")
    autor_scrape = obtener_autor_books_to_scrape("http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html")

    return {
        "Título": "No encontrado",
        "Autores": autor_scrape,
        "Descripción": "No disponible"
    }


# --- Ejemplo de uso ---
print(buscar_libro("A Light in the Attic"))


{'Título': 'A Light in the Attic', 'Autores': 'Shel Silverstein', 'Descripción': "NOW AVAILABLE AS AN EBOOK! From New York Times bestselling author Shel Silverstein, the creator of the beloved poetry collections Where the Sidewalk Ends, Falling Up, and Every Thing On It, comes an imaginative book of poems and drawings—a favorite of Shel Silverstein fans young and old. This digital edition also includes twelve poems previously only available in the special edition hardcover. A Light in the Attic delights with remarkable characters and hilariously profound poems in a collection readers will return to again and again. Here in the attic you will find Backward Bill, Sour Face Ann, the Meehoo with an Exactlywatt, and the Polar Bear in the Frigidaire. You will talk with Broiled Face, and find out what happens when Somebody steals your knees, you get caught by the Quick-Digesting Gink, a Mountain snores, and They Put a Brassiere on the Camel. Come on up to the attic of Shel Silverstein and let

In [88]:
# Función para obtener detalle completo de un libro
def obtener_detalle_libro(url_libro, categoria):
    try:
        response = requests.get(url_libro)
        soup = BeautifulSoup(response.text, "html.parser")
        
        titulo = soup.select_one(".product_main h1").get_text(strip=True)
        precio_text = soup.select_one(".price_color").get_text(strip=True)
        
        # Limpiar precio de manera más robusta
        try:
            precio_limpio = precio_text.replace('£', '').replace('Â', '').replace('€', '').strip()
            precio = float(precio_limpio)
        except ValueError:
            precio = 0.0  # Precio por defecto si no se puede convertir
            
        
        
        disponibilidad = soup.select_one(".instock.availability").get_text(strip=True)
        
        # Rating (está en class="star-rating X")
        rating_tag = soup.select_one(".star-rating")
        rating = rating_tag["class"][1] if rating_tag else "Sin rating"

        # UPC (está en la tabla de detalles)
        info_table = soup.select("table.table.table-striped tr")
        upc = info_table[0].td.get_text(strip=True) if len(info_table) > 0 else "Sin UPC"
        
        # Descripción del producto
        descripcion_tag = soup.select_one("#product_description ~ p")
        descripcion = descripcion_tag.get_text(strip=True) if descripcion_tag else "Sin descripción"

        # Buscar autor usando Google Books API
        info_google = buscar_libro(titulo)
        autor = info_google.get("Autores", "Autor desconocido")
        
        # Si Google no tiene descripción, usar la del sitio
        if info_google.get("Descripción") != "Sin descripción":
            descripcion = info_google.get("Descripción", descripcion)

        return {
            "titulo": titulo,
            "autor": autor,
            "precio": precio,
            "rating": rating,
            "disponibilidad": disponibilidad,
            "categoria": categoria,
            "link": url_libro,
            "UPC": upc,
            "descripcion": descripcion
        }
    except Exception as e:
        print(f"Error al obtener {url_libro}: {e}")
        return None

#  OBTENER LIBROS POR CATEGORIAS

In [93]:
import time # Para no hacer pedidos(requests) una tras otra, porque puede causar problemas

"""Creo una función y le doy dos argumentos"""
def obtener_libros_por_categoria(nombre_categoria,url_categoria):

    
    pagina_actual = url_categoria #Asigna la URL inicial de la categoría a una variable que iremos actualizando
    libros = [] # Lista vacía en donde se va guardar un diccionario por cada libro

    while True:
        print(f"Scrapeando {pagina_actual}...")
        respuesta = requests.get(pagina_actual)
        soup = BeautifulSoup(respuesta.text, "html.parser")

        libros_en_pagina = soup.select(".product_pod")
        
        for libro in libros_en_pagina:
            titulo = libro.h3.a["title"]
            precio = libro.select_one(".price_color").text.strip()
            rating = libro.p["class"][1]

            url_relativa = libro.h3.a["href"]
            url_libro = "/".join(pagina_actual.split("/")[:-1]) + "/" + url_relativa

            

            libros.append({
                "categoria": nombre_categoria,
                "titulo": titulo,
                "precio": precio,
                "rating": rating,
                "url_libro": url_libro,
               
            })


        next_button = soup.select_one("li.next a") # Busca si hay un enlace a la próxima página. Si existe, sigue
        # Construye la URL de la próxima página
        if next_button:
            next_url = next_button["href"]
            # Si pagina_actual = ".../index.html" y next_url = "page-2.html"➡ Reemplaza index.html con page-2.html
            if "index.html" in pagina_actual:
                pagina_actual = pagina_actual.replace("index.html", next_url )
            else:
                # Esta línea se ejecuta cuando hay más de una página en la categoría (ejemplo: page-2.html, page-3.html, etc.).
                pagina_actual = "/".join(pagina_actual.split("/")[:-1])+ "/" + next_url
        else: 
            break

        time.sleep(1) # Espera 1 segundo antes de pasar a la próxima página
    return libros # Devuelve la lista de diccionarios con todos los libros encontrados
#  lista vacía para juntar todos los libros del sitio
todos_los_libros = []
# Recorre todas las categorías que ya tenemos en el diccionario
for nombre,url in categoria.items():
    print(f"Scrapeando categoría: {nombre}...")

    libros_categoria = obtener_libros_por_categoria(nombre,url)

    todos_los_libros.extend(libros_categoria)  # agregamos todos los libros a la lista general
print(f"\nScraping completado. Total de libros encontrados: {len(todos_los_libros)}")

Scrapeando categoría: Travel...
Scrapeando https://books.toscrape.com/catalogue/category/books/travel_2/index.html...
Scrapeando categoría: Mystery...
Scrapeando https://books.toscrape.com/catalogue/category/books/mystery_3/index.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/mystery_3/page-2.html...
Scrapeando categoría: Historical Fiction...
Scrapeando https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-2.html...
Scrapeando categoría: Sequential Art...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-2.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-3.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-4.html...
Scrapeando cat

In [ ]:
# LIMPIAR Y GUARDAR LOS DATOS EN LA BASE DE DATOS
print("\n💾 Guardando datos en la base de datos...")

contador_guardados = 0

for libro in todos_los_libros:
    try:
        # Limpiar el precio
        precio_texto = libro['precio']
        precio_limpio = precio_texto.replace('£', '').replace('Â', '').strip()
        precio_float = float(precio_limpio)
        
        # Guardar categoría si no existe y obtener ID
        categoria_id = guardar_categoria(libro['categoria'])
        
        # Guardar libro en BD
        libro_id = guardar_libro(
            libro['titulo'],
            precio_float,  # Precio ya convertido a float
            libro['rating'],
            "In stock",  # Disponibilidad por defecto
            libro['url_libro'],
            "N/A",  # UPC no disponible en listado
            categoria_id,
            "Descripción no disponible"  # Descripción por defecto
        )
        
        # Para el autor, usar un nombre genérico por ahora
        # (podés mejorarlo después si querés)
        autor_id = guardar_autor("Autor por determinar")
        guardar_libro_autor(libro_id, autor_id)
        
        contador_guardados += 1
        
        if contador_guardados % 100 == 0:
            print(f"   📚 {contador_guardados} libros guardados...")
            
    except Exception as e:
        print(f"   ❌ Error guardando '{libro['titulo']}': {e}")
        continue

print(f"\n🎉 GUARDADO COMPLETADO!")
print(f"📊 {contador_guardados} libros guardados exitosamente")
print(f"🗃️ Base de datos 'libros.db' lista para las consultas")

#  EJECUCION DE SCRAPING COMPLETO

In [ ]:
# SCRAPING COMPLETO CON GUARDADO EN BASE DE DATOS
def ejecutar_scraping_completo():
    """
    Ejecuta el scraping completo del sitio y guarda todo en la base de datos
    """
    print("🕷️ Iniciando misión: Infiltración en Books To Scrape...")
    
    # Obtener todas las categorías
    categorias = extraer_categorias()
    
    contador_libros = 0
    
    # Recorrer cada categoría
    for nombre_categoria, url_categoria in categorias.items():
        print(f"\n📚 Scrapeando categoría: {nombre_categoria}")
        
        # Guardar categoría en BD y obtener su ID
        categoria_id = guardar_categoria(nombre_categoria)
        
        # Obtener libros básicos de la categoría
        libros_categoria = obtener_libros_por_categoria(nombre_categoria, url_categoria)
        
        # Procesar cada libro para obtener detalles completos
        for libro_basico in libros_categoria:
            print(f"  📖 Procesando: {libro_basico['titulo']}")
            
            # Obtener detalles completos del libro
            detalle = obtener_detalle_libro(libro_basico['url_libro'], nombre_categoria)
            
            if detalle:
                # Guardar libro en BD
                libro_id = guardar_libro(
                    detalle['titulo'],
                    detalle['precio'],
                    detalle['rating'],
                    detalle['disponibilidad'],
                    detalle['link'],
                    detalle['UPC'],
                    categoria_id,
                    detalle['descripcion']
                )
                
                # Procesar autores (pueden ser múltiples separados por coma)
                autores = detalle['autor'].split(', ')
                for autor_nombre in autores:
                    autor_nombre = autor_nombre.strip()
                    if autor_nombre and autor_nombre != "Autor desconocido":
                        # Guardar autor y relacionar con libro
                        autor_id = guardar_autor(autor_nombre)
                        guardar_libro_autor(libro_id, autor_id)
                
                contador_libros += 1
                
                # Pequeña pausa para no sobrecargar el servidor
                time.sleep(0.5)
        
        print(f"✅ Categoría {nombre_categoria} completada")
        time.sleep(1)  # Pausa entre categorías
    
    print(f"\n🎉 MISIÓN COMPLETADA!")
    print(f"📊 Total de libros procesados: {contador_libros}")
    print(f"🗃️ Base de datos 'libros.db' lista para la acción")

# Ejecutar el scraping completo
# ¡CUIDADO! Esto puede tomar bastante tiempo
ejecutar_scraping_completo()

# Las 5 consultas emocionales

In [ ]:
import sqlite3
import time

def ejecutar_consultas_emocionales():
    """
    Las 5 consultas que van a hacer llorar a la base de datos
    """
    conn = sqlite3.connect("libros.db")
    cursor = conn.cursor()
    
    print("🕵️ INVESTIGACIÓN: VERDADES OSCURAS DE LA LITERATURA")
    print("=" * 60)
    
    # 1. ¿Qué autor sigue publicando catástrofes de 1 estrella?
   
    print("Para cuando querés saber quién no debería publicar...")
    
    cursor.execute("""
        SELECT a.nombre, COUNT(*) as libros_horribles
        FROM autores a
        JOIN libro_autor la ON a.id = la.autor_id
        JOIN libros l ON la.libro_id = l.id
        WHERE l.rating = 'One'
        GROUP BY a.nombre
        HAVING COUNT(*) >= 3
        ORDER BY libros_horribles DESC
        LIMIT 5
    """)
    
    resultados = cursor.fetchall()
    for autor, cantidad in resultados:
        print(f"   😱 {autor}: {cantidad} desastres literarios")
    
    # 2. Capo de literatura barata pero bien rankeada
    print("\n💎 2. REY DE LA LITERATURA BARATA PERO BUENA")
    print("Cuando querés calidad sin quebrar la billetera...")
    
    cursor.execute("""
        SELECT a.nombre, AVG(l.precio) as precio_promedio, 
               COUNT(*) as total_libros,
               AVG(CASE l.rating 
                   WHEN 'Five' THEN 5
                   WHEN 'Four' THEN 4
                   WHEN 'Three' THEN 3
                   WHEN 'Two' THEN 2
                   WHEN 'One' THEN 1
                   ELSE 0 END) as rating_promedio
        FROM autores a
        JOIN libro_autor la ON a.id = la.autor_id
        JOIN libros l ON la.libro_id = l.id
        WHERE l.precio < 20
        GROUP BY a.nombre
        HAVING COUNT(*) >= 2 AND rating_promedio >= 4
        ORDER BY rating_promedio DESC, precio_promedio ASC
        LIMIT 10
    """)
    
    resultados = cursor.fetchall()
    for autor, precio_prom, total, rating_prom in resultados:
        print(f"   👑 {autor}: £{precio_prom:.2f} promedio, {total} libros, {rating_prom:.1f}⭐")
    
    # 3. Análisis por categoría: ¿Dónde está la plata?
    print("\n💰 3. CATEGORÍAS MÁS CARAS (Dónde se va tu plata)")
    print("Para saber en qué género vas a quedar en bancarrota...")
    
    cursor.execute("""
        SELECT c.nombre, 
               AVG(l.precio) as precio_promedio,
               MAX(l.precio) as precio_maximo,
               COUNT(*) as total_libros
        FROM categorias c
        JOIN libros l ON c.id = l.categoria_id
        GROUP BY c.nombre
        ORDER BY precio_promedio DESC
        LIMIT 10
    """)
    
    resultados = cursor.fetchall()
    for categoria, precio_prom, precio_max, total in resultados:
        print(f"   💸 {categoria}: £{precio_prom:.2f} promedio (máx: £{precio_max:.2f}) - {total} libros")
    
    # 4. Libros con rating alto pero baratos (las joyas ocultas)
    print("\n💍 4. JOYAS OCULTAS (Rating alto, precio bajo)")
    print("Para cuando querés calidad sin vender un riñón...")
    
    cursor.execute("""
        SELECT l.titulo, a.nombre as autor, l.precio, l.rating, c.nombre as categoria
        FROM libros l
        JOIN categorias c ON l.categoria_id = c.id
        LEFT JOIN libro_autor la ON l.id = la.libro_id
        LEFT JOIN autores a ON la.autor_id = a.id
        WHERE l.rating IN ('Four', 'Five') AND l.precio < 15
        ORDER BY l.rating DESC, l.precio ASC
        LIMIT 15
    """)
    
    resultados = cursor.fetchall()
    for titulo, autor, precio, rating, categoria in resultados:
        autor_display = autor if autor else "Autor desconocido"
        print(f"   💎 {titulo} - {autor_display} (£{precio:.2f}) [{rating}⭐] - {categoria}")
    
    # 5. Los autores más prolíficos
    print("\n🏭 5. FÁBRICAS DE LIBROS (Autores más prolíficos)")
    print("Para saber quién no para de escribir...")
    
    cursor.execute("""
        SELECT a.nombre, 
               COUNT(*) as total_libros,
               AVG(l.precio) as precio_promedio,
               AVG(CASE l.rating 
                   WHEN 'Five' THEN 5
                   WHEN 'Four' THEN 4
                   WHEN 'Three' THEN 3
                   WHEN 'Two' THEN 2
                   WHEN 'One' THEN 1
                   ELSE 0 END) as rating_promedio
        FROM autores a
        JOIN libro_autor la ON a.id = la.autor_id
        JOIN libros l ON la.libro_id = l.id
        GROUP BY a.nombre
        HAVING COUNT(*) >= 3
        ORDER BY total_libros DESC, rating_promedio DESC
        LIMIT 10
    """)
    
    resultados = cursor.fetchall()
    for autor, total, precio_prom, rating_prom in resultados:
        print(f"   📚 {autor}: {total} libros, £{precio_prom:.2f} promedio, {rating_prom:.1f}⭐")
    
    conn.close()

def demostrar_indexacion():
    """
    Demostración del poder de los índices: antes y después
    """
    print("\n⚡ DEMOSTRACIÓN DE INDEXACIÓN: EL PODER DE LA VELOCIDAD")
    print("=" * 60)
    
    conn = sqlite3.connect("libros.db")
    cursor = conn.cursor()
    
    # Consulta compleja SIN índice
    consulta_lenta = """
        SELECT l.titulo, a.nombre, l.precio, c.nombre
        FROM libros l
        JOIN libro_autor la ON l.id = la.libro_id
        JOIN autores a ON la.autor_id = a.id
        JOIN categorias c ON l.categoria_id = c.id
        WHERE l.precio > 30 AND l.rating IN ('Four', 'Five')
        ORDER BY l.precio DESC
    """
    
    print("\n🐌 ANTES: Sin índices (preparate para el dolor)")
    start_time = time.time()
    cursor.execute(consulta_lenta)
    resultados_sin_indice = cursor.fetchall()
    tiempo_sin_indice = time.time() - start_time
    print(f"   Tiempo sin índices: {tiempo_sin_indice:.4f} segundos")
    print(f"   Resultados encontrados: {len(resultados_sin_indice)}")
    
    # Crear índices estratégicos
    print("\n🔧 Creando índices mágicos...")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_libros_precio ON libros(precio)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_libros_rating ON libros(rating)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_libro_autor_libro ON libro_autor(libro_id)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_libro_autor_autor ON libro_autor(autor_id)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_libros_categoria ON libros(categoria_id)")
    
    print("\n🚀 DESPUÉS: Con índices (siente el poder)")
    start_time = time.time()
    cursor.execute(consulta_lenta)
    resultados_con_indice = cursor.fetchall()
    tiempo_con_indice = time.time() - start_time
    print(f"   Tiempo con índices: {tiempo_con_indice:.4f} segundos")
    print(f"   Resultados encontrados: {len(resultados_con_indice)}")
    
    if tiempo_sin_indice > 0:
        mejora = ((tiempo_sin_indice - tiempo_con_indice) / tiempo_sin_indice) * 100
        print(f"   🎯 Mejora de rendimiento: {mejora:.1f}% más rápido")
    
    # Mostrar algunos resultados
    print("\n📋 Algunos libros caros pero buenos encontrados:")
    for i, (titulo, autor, precio, categoria) in enumerate(resultados_con_indice[:5]):
        print(f"   {i+1}. {titulo} - {autor} (£{precio:.2f}) [{categoria}]")
    
    conn.close()

# Ejecutar las consultas
ejecutar_consultas_emocionales()
demostrar_indexacion()